Обозначения:

📝 - определение, которое нужно запомнить

`# ⏩`- комментарий, что эту ячейку / функцию нужно просто запустить, она уже написана

`# ✏️` - код в этой ячейке / функции мы будем дополнять в процессе урока

👨🏻‍💻 - задача для самостоятельного выполнения

[Чистая рабочая тетрадь для заполнения](https://colab.research.google.com/drive/1hQ69Nn3nnCF4PpBN2HOxXXDJQQm-mL0i?usp=sharing)

[Заполненная рабочая тетрадь](https://colab.research.google.com/drive/1DOIDJXYM9a-DAGGnHIK4qyZECwg0higW?usp=sharing)

[Презентация к уроку](https://docs.google.com/presentation/d/10m7FYPHtHWrby_vl6Z1a-pkrL8magcM0wIruskHNka4/edit?usp=sharing)

# Чем мы будем заниматься на уроке?

Мы напишем систему для подбора похожих новостей. Возьмем данные о новостях с Lenta.ru, с помощью нейросети Word2Vec превратим текстовые описания в числа и объединим в группы на основе близости чисел.

# Немного теории: концептуальная схема работы нейронной сети

Вспомним, в чем заключается задача Data Scientist:
<table>
  <tr>
    <th></th> <th>Прямая задача</th> <th>Обратная задача</th>
  </tr>
  <tr>
    <th>Дано</th>
    <td>y = f(x)</td>
    <td bgcolor="#93E9BE">Координаты y</td>
  </tr>
  <tr>
    <th>Найти</th>
    <td>Координаты y</td>
    <td bgcolor="#93E9BE">Функцию y = z(x)</td>
  </tr>
  <tr>
    <th></th>
    <th>↑ Этому учат в школе ↑</th>
    <th>↑ <u>Этим занимаются DS</u>↑</th>
  </tr>
</table>

📝 **Нейронная сеть** - тип модели машинного обучения, которая состоит из комбинации нейронов - простых функций вида $y_k = w_1 x_2 + ... + w_i x_i$.

📝 **Веса модели** - коэффициенты $w_1 , \dots,  x_i$

Крутость нейросети в том, с помощью нейронов мы можем приблизительно вычислить практически любую функцию (аналогия: цифровое фото из квадратных пикселей). Любая модель семейства GPT по сути являются функцией, описывающей текст на человеческом языке, т.н. "большая языковая модель".

Один из простейших примеров - вычисление функции XOR(a, b) - исключающего ИЛИ. Аргументы a и b могут быть равны 1 или 0. XOR(a, b) вычисляется по следующему правилу:
* XOR = 0, если a = b,
* XOR = 1, если a != b.

Нейронная сеть будет вычислять эту функцию по следующей схеме:

<table style="border: 1px solid transparent">
    <tr><td><img src='https://drive.google.com/uc?export=view&id=1zn68tWxZpZzWOrtxlL4_MHy8MWFIyrEE' height=200>
    </td></tr>
    <tr><td>Картинка с <a href="https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD">Википедии</a></td></tr>
</table>


Сложность работы с нейронной сетью - это подбор правильных $w_i$. Для этого используются разные комбинации нейронов и разные типы задач.

Например, чтобы преобразовать текст в числа, модель обучают угадывать замаскированное слово по контексту, а потом берут получившиеся веса.
Например, "Мороз и солнце, [???] чудесный". После получения представления текста в виде чисел из весов модели, мы можем выполнять различные математические преобразования над текстом, например "король − мужчина + женщина = королева" (можно подробнее почитать на [Хабре](https://habr.com/ru/articles/446530/))

# Практика - подбираем релевантные новости с помощью нейросети

Пишем систему для рекомендации похожих новостей.

Наш датасет с новостями имеет темы - **topic** ("Спорт", "Наука и техника", ...) и тэги **tag** ("Футбол", "Техника", "Космос", ...).

1. Возьмем новости, помеченные одним тэгом и разобьем на еще более маленькие группы по похожести текстов.

2. Используем библиотеку, реализующую нейросеть архитектуры Word2Vec от [gensim](https://pypi.org/project/gensim/) и готовые веса для нее от [RusVectores](https://rusvectores.org/ru/models/). Преобразуем тексты в наборы чисел.

3. Используем модели для 📝 **кластеризации** - разбиения набора данных на группы наиболее похожих элементов. Воспользуемся реализацией из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html).

In [ ]:
# ▶️▶️
# Подключаем нужные библиотеки для визуализации

import nltk
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans

nltk.download('punkt') # загружаем дополнительные данные для библиотеки разбиения текстов на слова

In [ ]:
# ▶️▶️
# Настраиваем отображение таблиц
from IPython.display import display
pd.set_option('display.max_colwidth', 200)

In [ ]:
# ▶️▶️
# Скачиваем библиотеку для загрузки файлов
!pip install wget --quiet

Команда %%time в начале ячейки замеряет время ее выполнения

In [ ]:
%%time
# ▶️▶️
# Выполняется ≈1 минуту

# Скачиваем веса модели

from wget import download

download('https://rusvectores.org/static/models/rusvectores4/news/news_upos_cbow_600_2_2018.vec.gz',
         'news_upos_cbow_600_2_2018.vec.gz')

In [ ]:
%%time
# ▶️▶️
# Выполняется ≈2-3 минуты
word2vec = KeyedVectors.load_word2vec_format('news_upos_cbow_600_2_2018.vec.gz')

In [ ]:
# ✏️
# Атрибут index_to_key объекта word2vec содержит список слов,
# на котором обучалась модель

...

У нашей модели есть особенность: для ее обучения к словам цепляли [тэги с частью речи](https://universaldependencies.org/u/pos/all.html), например, 'год_NOUN', 'сообщать_VERB'.

Чтобы сопоставить обычное слово и слово с тэгом из модели, составим словарь переименования.

In [ ]:
# ▶️▶️
# Метод split разбивает слово на список подслов по заданному символу

'год_NOUN'.split('_')

In [ ]:
# ▶️▶️
# В квадратных скобках из списка получаем начальный элемент (нумерация с 0)
'год_NOUN'.split('_')[0]

In [ ]:
# ✏️
# Создадим словарь вида {слово: слово_ТЭГ}.
# В конце списка встречаются некорректные тэги, например, 'год_NOUN' 'год_PROPN'
# или ['давать_VERB', 'давать_NOUN', 'давать_ADJ', 'давать_PROPN', 'давать_NUM']
# Поэтому будем брать только первую версию слова, и если оно уже есть в словаре,
# перезаписывать его не будем

lemma2word = {}

for tagged_word in word2vec.index_to_key:
    word = ...
    ...

In [ ]:
# ▶️▶️
lemma2word['год']

# Векторизация

Для того, чтобы иметь возможность автоматически анализировать данные, нужно придать тексту числовую структуру. Для этого сделаем векторизацию. Нам нужно пройти несколько стадий: токенизация → лемматизация → векторизация.

📝 **Токенизация** - процесс разбиения текста на слова (**токен** - минимальная единица язчка, имеющая смысл)

👨🏻‍💻 Вопрос: можно ли назвать букву токеном?

📝 **Лемматизация** - получение начальной формы слова: "годы", "году" → "год"

📝 **Векторизация** - процесс превращения текста в набор чисел - **векторное представление**. Например, "год" → (0.1, 0, 1.2, 3)

📝 **Вектор** - набор чисел, например, (0.1, 0, 1.2, 3). Над векторами можно проводить арифметические операции аналогично числам - например, попарное сложение элементов: (1, 2, 5) + (3, 4, 0) = (4, 6, 5) или вычисление расстояния между ними.

На основе близости векторных представлений текстов мы и будем искать похожие новости.

In [ ]:
# ▶️▶️
# Пример токенизации

word_tokenize('Мороз и солнце, день чудесный', language='russian')

In [ ]:
# ▶️▶️
# mystem - инструмент для лемматизации
mystem = Mystem()

In [ ]:
# ▶️▶️
# Функция получения тэгированной формы слова из модели Word2Vec

def get_w2v_word(word):
    # Получаем начальную форму слова
    lemma = mystem.lemmatize(word)[0]
    # Берем тэгированную форму из составленного ранее словаря
    w2v_word = lemma2word.get(lemma)
    return w2v_word

In [ ]:
# ▶️▶️
get_w2v_word("Мороз")

In [ ]:
# ▶️▶️
# Функция получения тэгированной формы слова из модели Word2Vec

def vectorize_sentence(txt):
    words = word_tokenize(txt, language='russian')
    vectors = []
    for word in words:
        tagged_word = get_w2v_word(word)
        if tagged_word is not None:
            vector = word2vec[tagged_word]
            vectors.append(vector)
    return np.mean(vectors, axis=0)

In [ ]:
# ▶️▶️
vectorize_sentence('Мороз и солнце, день чудесный')

# Возьмем данные о новостях с Lenta.ru

In [ ]:
# ▶️▶️
data = pd.read_csv('https://raw.githubusercontent.com/anastasiarazb/skillbox_nlp_demo/master/lenta_example.csv',
                   sep=',')
data

In [ ]:
# ✏️
# Выведем списки тем и тэгов со счетчиком числа статей в этих группах
# Функция groupby по списку колонок ['topic', 'tags'] группирует данные,
# а count() - дает количество записей в каждой группе


In [ ]:
# ✏️
# В теме 'Экономика' возьмем тэг 'undefined' и исследуем новости в нем

data_tag = data[
    (data['topic'] == ...)
    & (data['tags'] == ...)
    ].copy()

# 👨🏻‍💻 Попробуйте другую комбинацию темы и тэга, напр. "Наука и техника"+"Техника"

data_tag.shape

In [ ]:
# ✏️

# Представим заголовки и тексты в виде чисел с помощью
# объединения векторных представлений, полученных с помощью
# vectorize_sentence для каждого в отдельности
# Операция + для двух списков объединяет их: [1, 2] + [3, 4] = [1, 2, 3, 4]

vectors = []

for title, text in tqdm(data_tag[['title', 'text']].values):
    vector = ...
    vectors.append(vector)

# 👨🏻‍💻 Попробуйте использовать только 1 элемент статьи, только текст
#    или только заголовок
#    (для этого в цикле надо будет оставить в vector, например,
#    только vectorize_sentence(text))

vectors = np.array(vectors)

# Кластеризация

📝 **Кластеризация** - объединение похожих объектов в группы.

Есть различные методы кластеризации, мы будем пользоваться библиотекой [sklearn](https://scikit-learn.org/stable/modules/clustering.html)

In [ ]:
# ▶️▶️

from sklearn.cluster import SpectralClustering, KMeans

clusterizer = SpectralClustering(n_clusters=8, n_init=10, random_state=42)

# 👨🏻‍💻 Попробуйте другую модель кластеризации, например, KMeans.
#    Описание различных методов: https://scikit-learn.org/stable/modules/clustering.html

# 👨🏻‍💻 Попробуйте заменить параметры, например, установить n_clusters=10.
#    Есть ли какие-то идеи, как правильно выбрать количество кластеров?

clusterizer.fit(vectors)

In [ ]:
# ✏️
# Добавим разметку с номером кластера clusterizer.labels_
# как новую колонку в данные 'label' в таблицу data_tag


In [ ]:
# ✏️

# Набор меток data_tag['label'].unique() отсортируем через sorted()
# и для каждой метки выведем label и подтаблицу строк с этой
# меткой кластера data_tag[data_tag['label'] == label]

for ... :
    print(...)
    display(...)

# Что мы узнали на этом занятии?

* ML Engineer - это специалист, который строит модели машинного обучения, то есть системы, которые создают "магию" интеллектуального поведения

* Это творческая профессия, где каждый случай нужно исследовать

* Мы разобрали, как работает нейронная сеть

* Научились переводить человеческий текст в численное представление (векторы) и объединять их в группы с помощью методов кластеризации

* Все необходимые для работы функции уже реализованы в библиотеках. Мы познакомились с библиотеками gensim, nltk, pymystem, sklearn